In [1]:
import pandas as pd
import my_file
import sqlalchemy
import requests

In [2]:
schema="gans"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password=my_file.mysql_password # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [3]:
cities_df = pd.read_sql_table('cities',con=con)
cities_df

,city_id,City,Country,Population,latitude,longitude
0,1,Berlin,DE,3800000,52.3112,13.2418
1,2,London,GB,4100000,51.3026,0.7390


In [4]:
API_key = my_file.weather_api_key

In [5]:
for index, row in cities_df.iterrows():
    print(index)
    print(row['City'])
    print(row['city_id'])
    print('NEW_ITERATION')

0
Berlin
1
NEW_ITERATION
1
London
2
NEW_ITERATION


In [6]:
list_of_jsons = []

for index, row in cities_df.iterrows():
    
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={row['City']}&appid={API_key}&units=metric")

    response = requests.get(url)
    list_of_jsons.append(response.json())

In [8]:
from IPython.display import JSON
JSON(list_of_jsons[0])

<IPython.core.display.JSON object>

In [10]:
weather_dict = {'city_id':[],
                'forecast_time': [],
                'temperature': [],
                'wind_speed': [],
                'humidity': []}

for index, row in cities_df.iterrows():
    for weather in list_of_jsons[index]['list'][:9]:
        weather_dict['city_id'].append(row['city_id'])
        weather_dict['forecast_time'].append(weather['dt_txt'])
        weather_dict['temperature'].append(weather['main']['temp'])
        weather_dict['wind_speed'].append(weather['wind']['speed'])
        weather_dict['humidity'].append(weather['main']['humidity'])
        
weather_df=pd.DataFrame(weather_dict)
weather_df

,city_id,forecast_time,temperature,wind_speed,humidity
0,1,2023-09-06 15:00:00,29.34,2.87,39
1,1,2023-09-06 18:00:00,27.74,2.68,41
2,1,2023-09-06 21:00:00,24.35,2.71,51
3,1,2023-09-07 00:00:00,19.97,2.34,59
4,1,2023-09-07 03:00:00,18.40,2.51,62
5,1,2023-09-07 06:00:00,19.10,2.89,58
6,1,2023-09-07 09:00:00,25.30,3.82,36
7,1,2023-09-07 12:00:00,29.00,4.53,27
8,1,2023-09-07 15:00:00,29.03,4.58,27
9,2,2023-09-06 15:00:00,30.22,0.77,51


In [11]:
weather_df.to_sql('weathers',if_exists='append',index=False,con=con)

18

In [13]:
# weather_dict = {'city_id':[],
#                 'forecast_time': [],
#                 'temperature': [],
#                 'wind_speed': [],
#                 'humidity': []}

# for index, row in cities_df.iterrows():
    
#     url = (f"http://api.openweathermap.org/data/2.5/forecast?q={row['City']}&appid={API_key}&units=metric")

#     response = requests.get(url)
#     my_json = response.json()
#     for weather in my_json['list'][:9]:
#         weather_dict['city_id'].append(row['city_id'])
#         weather_dict['forecast_time'].append(weather['dt_txt'])
#         weather_dict['temperature'].append(weather['main']['temp'])
#         weather_dict['wind_speed'].append(weather['wind']['speed'])
#         weather_dict['humidity'].append(weather['main']['humidity'])
        
# weather_df=pd.DataFrame(weather_dict)
# weather_df